# Team EEG Data
Jungsu Pak, Rufei Fan, Alice Wong, Aaron Gonzales, Nathan Ong, Andrew Lee

In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

## Loading Data
The cells below are for to you use to load the data in case they are similar to loading a specific series. Feel free to disregard them if you load the data another way.

In [2]:
def load_task_eeg_series(datapath, sub, series):
    # input arguments:
    # datapath (string): path to the root directory
    # sub (string): subject ID (e.g. subj1, subj2, etc)
    # series (int): series name (e.g. 1, 2, etc). 
    # This will load in all of the specified data and chunk them by series
    
    # output:
    # eegdata (numpy array): samples x channels data matrix
    # eegevents (pandas dataframe): labels
    # channel_names (list): names of the channels
    import os
    import pandas as pd
    eegdata = pd.read_csv(os.path.join(datapath, sub + '_series' + str(series) + '_data.csv'))
    eegevents = pd.read_csv(os.path.join(datapath, sub + '_series' + str(series) + '_events.csv'))
    return eegdata.iloc[:].as_matrix(), eegevents, eegdata.keys()

def load_task_eeg_data(datapath, sub, start, end):
    # call this one in your code
    # input arguments:
    # datapath (string): path to the root directory
    # sub (string): subject ID (e.g. subj1, subj2, etc)
    # series (list): series number (e.g. [1,2,3] etc). 
    # This will load in all of the specified data and chunk them by series
    
    # output:
    # eegdata (numpy array): samples x channels data matrix
    # eegevents (pandas dataframe): labels
    # channel_names (list): names of the channels
    import pandas as pd
    import numpy as np
    eegdata = []
    eegevents = []
    for s in range(start, end): # end is not inclusive
        ed, ee, ek = load_task_eeg_series(datapath, sub, s)
        ee['chunks'] = pd.Series(s * np.ones(ee.shape[0]))
        eegdata.append(ed)
        eegevents.append(ee)
    eegkeys = ek    
    return np.vstack(eegdata), pd.concat(eegevents), eegkeys

def load_data(subj, train_start, train_end, test_start, test_end):
    # subj requires a string (i.e. 'subj1')
    # train_start, train_end is a range that specifies which series you want as training
    #     (1, 7) means that the series from 1 to 6 (inclusive) are used for training
    # test_start, test_end is also a range
    #     (7, 9) means that the series from 7 to 8 (inclusive) are returned for testing
    data_path = 'train' # modify this
    train_data, train_event, tmp = load_task_eeg_data(data_path, subj, train_start, train_end)
    test_data, test_event, tmp = load_task_eeg_data(data_path, subj, test_start, test_end)
    return train_data, train_event, test_data, test_event

## How we processed our data
This is case you want to use it. We used this for training our model. 

x is expected to be filtered and scaled (preprocessed) before calling this:

```
binary_shrink_data(x, y, nonlabel_factor=0.5, reduction_factor=3, shuffle_indices=False, verbose=True)
```

178803 rows with something and 1243589 rows with nothing.

178803 rows with something and 177656 rows with nothing.

Returning 71292 samples.


What's happening it looks at the labels and count how many are 0s and 1s. In all series the number of samples with no label dominates the number of samples with some label. It shrinks the data by getting rid of samples with no labels until they the no-label samples account for `nonlabel_factor` % of the data. Then it naively downsamples the data by a factor of `reduction_factor`, by taking every n sample.

In [3]:
def calc_reduction_ratio(a, b, factor):
    # helper method
    # it returns the factor to scale data to make them proportional by a factor of factor (var)
    n = (a - factor * a) / float(factor * b)
    return int(n)

def reduce_samples_by_factor(x, y, factor):
    # helper method that naively downsamples data without worrying about aliasing
    return x[::factor], y[::factor]

def binary_reduce_empty_samples(x, y, reduction_factor, shuffle_indices=False, verbose=False):
    # x: np array, expects n x 32
    # y: np array, expects n x 6
    somethings = []
    nothings = []
    
    new_y = np.zeros(y.shape[0], dtype=float)
    
    for i in range(y.shape[0]):
        sum = np.sum(y[i, :])
        if sum > 0:
            somethings.append(i)
            new_y[i] = 1.
        else:
            nothings.append(i)
            new_y[i] = 0.
            
    if verbose:
        print('%d rows with something and %d rows with nothing.' % (len(somethings), len(nothings)))
        
    factor = calc_reduction_ratio(len(nothings), len(somethings), reduction_factor)
    
    if shuffle_indices:
        nothings = shuffle(nothings)

    nothings = nothings[::factor] # take 1 out of every n to reduce the amount of nothings
    
    if verbose:
        print('%d rows with something and %d rows with nothing.' % (len(somethings), len(nothings)))
    
    all_things = sorted(somethings + nothings)

    return x[all_things], new_y[all_things]

def binary_shrink_data(x, y, nonlabel_factor, reduction_factor, shuffle_indices=False, verbose=False):
#     nonlabel_factor = 0.5 # nonlabel samples will account for % of the total data
#     reduction_factor = 50 # divide number of samples by reduction factor
    X_bal, y_bal = binary_reduce_empty_samples(x, y, nonlabel_factor, shuffle_indices, verbose)
    X_bal, y_bal = reduce_samples_by_factor(X_bal, y_bal, reduction_factor)
    if verbose:
        print 'Returning %d samples.' % (X_bal.shape[0])
    return X_bal, y_bal

## Preprocessing
These are the methods used for preprocessing. You won't need to preprocess the data manually, although you will need to preprocess the labels it is not sure how you (Jeff) are loading them.

During class today Uri said you are testing it by reducing the sampling frequency from 500 Hz to something smaller. You will need to do this in the `preprocess_data` method because if you do it before the filter you may get unexpected results.

In [4]:
# Preprocessing for skl models

def apply_filters(x, order=4, fs=500.0, cutoff=50, axis=0):
    # to use this just pass the data and use the existing data
    # expects n x 32 in sequence
    
    # x: data, whole thing
    # fs: frequency, 500
    from scipy.signal import decimate, butter, filtfilt
    nyq = .5 * fs
    b, a = butter(order, cutoff/nyq, btype='low')
    x = filtfilt(b, a, x, axis=axis)
    return x

def preprocess_data(X_orig, scaler):
    # X_orig: eeg data, np array that is n x 32
    # scaler for the subject
    X_prep = scaler.transform(X_orig)
    X_prep = apply_filters(X_prep)
    
    # May need to add code here
    # if you are simply reducing the sampling frequency you can downsample by:
    # X_prep = X_prep[::downsample_factor]
    
    return X_prep
    
def preprocess_labels(y):
    # takes a n x 6 np array
    # returns a n x 1 np array
    new_y = np.zeros(y.shape[0], dtype=float)
    
    for i in range(y.shape[0]):
        sum = np.sum(y[i, :])
        if sum > 0:
            new_y[i] = 1.
        else:
            new_y[i] = 0.
    
    return new_y

In [8]:
# Preprocessing for Keras model
# No scaling used in it, just filtering with bandpass

def butter_bandpass(lowcut, highcut, fs=500, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs=500, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data, axis = 0)
    return y

def keras_preprocess_data(X_orig):
    return butter_bandpass_filter(X_orig, 1, 50, 500, order=5)

# Run a Model

## Skl Model
Call `run_skl_model` using the raw data, it expects a n x 32 numpy array with data type being float64, although StandardScaler may automatically convert it for you. 

How to use: `run_skl_model(x, subj_id=1)` runs it for subject 1. It handles the preprocessing of the data for you.
It will output a 1 if it expects any label in it. In our experience it shows a high amount of false positives, falsely classifying 0s as 1s.

As mentioned in comments we only provided the models for subjects 1, 4, 7, and 10 for skl models.

## Keras Model

The Keras model is for subject 3. After you load your data for subject 3 call:

```
x = keras_preprocess_data(x_data)
run_keras_model(x)
```

It will output either 0 or 1.

In [6]:
def run_skl_model(x, subj_id):
    # subj_id: id of the subject
    # we have models for subjects 1, 4, 7, & 10
    # choosing an alternate will not load
    # this preprocesses data for you but not labels
    subj_str = 'subj%d' % (subj_id)
    scaler_str = '%s_scaler.pkl' % (subj_str)
    model_str = '%s_model.pkl' % (subj_str) 
    
    scaler = joblib.load(scaler_str)
    clf = joblib.load(model_str)
    
    proc_x = preprocess_data(x, scaler)

    return clf.predict(x)

In [9]:
def run_keras_model(x):
    # expects x to be preprocessed using keras_preprocess_data 
    from keras.models import load_model
    model = load_model('KerasNNet.h5')  # load the saved keras model (and weights) so you don't have to retrain
    prediction = model.predict_classes(x)
    return prediction